# Libraries

In [1]:
import os
import glob
import itertools
import random
import math
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset

from src.utils import set_seed, print_h, check_dataset_info, get_vgrf_window_data

# Config

In [ ]:
seed = 69
set_seed(seed)

data_dir = 'data'
vgrf_data_dir = data_dir + '/gait-in-parkinsons-disease-1.0.0'
demographics_data_path = vgrf_data_dir + '/demographics.xls'

n_feat = 16
window_size = 500
stride_size = 250
min_vgrf_data_len = (5-1)*stride_size + window_size
max_vgrf_data_len = 25_000
n_class = 4
k_fold = 10
test_person_split = 0.15
test_window_split = 0.15
val_person = True